In [1]:
import numpy as np

def weighted_dtw(X, Y, lam, q):
    """
    計算兩個多變量時間序列 X 與 Y 的加權 DTW 距離，並回傳最短路徑
    X: np.array, shape (Lx, d)
    Y: np.array, shape (Ly, d)
    lam: np.array, shape (d, )，維度權重
    q: 超參數 (q>1)
    """
    Lx, d = X.shape
    Ly = Y.shape[0]
    
    # 計算點對點加權歐氏距離 cost[i,j] = sum_d (lam[d]^q * (X[i,d]-Y[j,d])^2)
    cost = np.zeros((Lx, Ly))
    for i in range(Lx):
        for j in range(Ly):
            diff = X[i] - Y[j]
            cost[i, j] = np.sum((lam**q) * (diff**2))
    
    # 動態規劃計算 DTW 累積距離矩陣
    dp = np.full((Lx, Ly), np.inf)
    dp[0, 0] = cost[0, 0]
    for i in range(1, Lx):
        dp[i, 0] = cost[i, 0] + dp[i-1, 0]
    for j in range(1, Ly):
        dp[0, j] = cost[0, j] + dp[0, j-1]
    for i in range(1, Lx):
        for j in range(1, Ly):
            dp[i, j] = cost[i, j] + min(dp[i-1, j], dp[i, j-1], dp[i-1, j-1])
    
    # 回溯獲取最短扭曲路徑
    path = []
    i, j = Lx - 1, Ly - 1
    path.append((i, j))
    while i > 0 or j > 0:
        if i == 0:
            j -= 1
        elif j == 0:
            i -= 1
        else:
            idx = np.argmin([dp[i-1, j], dp[i, j-1], dp[i-1, j-1]])
            if idx == 0:
                i -= 1
            elif idx == 1:
                j -= 1
            else:
                i -= 1
                j -= 1
        path.append((i, j))
    path.reverse()
    return dp[-1, -1], path

class FCM_wDTW:
    def __init__(self, c=3, m=1.5, q=2.0, epsilon=1e-3, max_iter=50):
        """
        c: 聚類數量
        m: 模糊係數
        q: 權重指數 (q>1)
        epsilon: 收斂閾值
        max_iter: 最大迭代次數
        """
        self.c = c
        self.m = m
        self.q = q
        self.epsilon = epsilon
        self.max_iter = max_iter

    def fit(self, data):
        """
        data: list of np.array，每個元素 shape 為 (L, d)
        """
        self.n = len(data)            # 樣本數
        self.data = data
        self.L = data[0].shape[0]      # 假設所有樣本長度相同
        self.d = data[0].shape[1]
        
        # 1. 初始化聚類中心 V：隨機選取 c 個樣本作為初始中心
        indices = np.random.choice(self.n, self.c, replace=False)
        self.V = [data[i].copy() for i in indices]
        
        # 2. 隨機初始化權重 lam，滿足 sum(lam)=1
        self.lam = np.random.rand(self.d)
        self.lam = self.lam / np.sum(self.lam)
        
        # 3. 初始化會員矩陣 U (shape: c x n)，對每個樣本 j, sum_i u[i,j]=1
        self.U = np.random.rand(self.c, self.n)
        self.U = self.U / np.sum(self.U, axis=0, keepdims=True)
        
        # 儲存每個 (i,j) 對應的扭曲路徑
        self.warping_paths = dict()
        
        prev_J = np.inf
        for iteration in range(self.max_iter):
            # ----------------------------
            # Problem 1: 更新最優扭曲路徑 (OWP)
            # ----------------------------
            D = np.zeros((self.c, self.n))
            for i in range(self.c):
                for j in range(self.n):
                    dist, path = weighted_dtw(self.V[i], data[j], self.lam, self.q)
                    D[i, j] = dist
                    self.warping_paths[(i, j)] = path
                    
            # ----------------------------
            # Problem 2: 更新會員矩陣 U (根據公式 (4))
            # ----------------------------
            for j in range(self.n):
                for i in range(self.c):
                    denom = 0.0
                    for s in range(self.c):
                        # 避免除以零，加入小常數
                        ratio = D[i, j] / (D[s, j] + 1e-10)
                        denom += ratio**(1 / (self.m - 1))
                    self.U[i, j] = 1.0 / (denom + 1e-10)
            
            # ----------------------------
            # Problem 3: 更新權重 Λ (根據公式 (8))
            # ----------------------------
            A = np.zeros(self.d)
            for d_idx in range(self.d):
                for i in range(self.c):
                    for j in range(self.n):
                        path = self.warping_paths[(i, j)]
                        sum_sq = 0.0
                        for (r, s) in path:
                            diff = self.V[i][r, d_idx] - data[j][s, d_idx]
                            sum_sq += diff**2
                        A[d_idx] += (self.U[i, j]**self.m) * sum_sq
            new_lam = np.zeros(self.d)
            for d_idx in range(self.d):
                denom = 0.0
                for s in range(self.d):
                    denom += (A[d_idx] / (A[s] + 1e-10))**(1 / (self.q - 1))
                new_lam[d_idx] = 1.0 / (denom + 1e-10)
            # 正規化權重
            new_lam = new_lam / np.sum(new_lam)
            self.lam = new_lam
            
            # ----------------------------
            # Problem 4: 更新聚類中心 V (根據公式 (9))
            # ----------------------------
            new_V = []
            for i in range(self.c):
                # 初始化新的中心，形狀 (L, d)
                v_new = np.zeros((self.L, self.d))
                count = np.zeros((self.L, 1))
                for j in range(self.n):
                    path = self.warping_paths[(i, j)]
                    u_m = self.U[i, j]**self.m
                    # 對每個對齊對 (r, s)，累計來自樣本 x_j 的值
                    for (r, s) in path:
                        v_new[r] += u_m * data[j][s]
                        count[r] += u_m
                # 防止除以0
                for r in range(self.L):
                    if count[r] > 0:
                        v_new[r] /= count[r]
                    else:
                        v_new[r] = self.V[i][r]
                new_V.append(v_new)
            self.V = new_V
            
            # ----------------------------
            # 計算目標函數 J (根據公式 (2))
            # ----------------------------
            J = 0.0
            for i in range(self.c):
                for j in range(self.n):
                    J += (self.U[i, j]**self.m) * D[i, j]
            print("Iteration", iteration, "Objective J =", J)
            if abs(prev_J - J) < self.epsilon:
                print("Converged at iteration", iteration)
                break
            prev_J = J
        self.J = J

    def reconstruct(self, x):
        """
        根據學到的隱空間重構單一樣本 x (公式 (11))
        這裡對每個聚類中心 i 計算與 x 的扭曲路徑，然後加權平均
        """
        # 初始化重構結果，形狀同 x
        recon = np.zeros_like(x)
        weight_sum = np.zeros((x.shape[0], 1))
        # 為了簡化，這裡對各聚類中心的貢獻以 U 的平均值近似
        for i in range(self.c):
            dist, path = weighted_dtw(self.V[i], x, self.lam, self.q)
            # 這裡取該聚類中心對所有樣本的平均會員值作為加權
            u_m = np.mean(self.U[i, :])**self.m
            for (r, s) in path:
                recon[s] += u_m * self.V[i][r]
                weight_sum[s] += u_m
        for s in range(x.shape[0]):
            if weight_sum[s] > 0:
                recon[s] /= weight_sum[s]
            else:
                recon[s] = x[s]
        return recon

    def anomaly_score(self, x):
        """
        根據公式 (12)，計算樣本 x 的異常分數，即 x 與其重構樣本之 wDTW 距離
        """
        x_recon = self.reconstruct(x)
        score, _ = weighted_dtw(x, x_recon, self.lam, self.q)
        return score

# ----------------------------
# 測試與模擬
# ----------------------------
def simulate_test_data(length=50, d=3):
    """
    生成 n_samples 筆模擬多變量時間序列數據
    每筆數據為長度 length、維度 d 的時間序列
    """
    serie = np.sin(np.linspace(0, 4*np.pi, 100)) + np.random.normal(0, .2, size = (100))
    data = []
    for i in range(length, serie.shape[0]):
        data.append(serie[i-length:i].reshape(-1, d))
    data.append(np.random.normal(size = (length, d)))
    return data

if __name__ == "__main__":
    # 模擬 20 筆測試數據
    data = simulate_test_data(length=50, d=1)
    
    # 建立並訓練 FCM-wDTW 模型
    model = FCM_wDTW(c=3, m=1.5, q=2.0, epsilon=1e-3, max_iter=10)
    model.fit(data)
    
    # 計算每筆樣本的異常分數
    scores = []
    for i, x in enumerate(data):
        score = model.anomaly_score(x)
        scores.append(score)
        print("Sample", i, "anomaly score:", score)
    print("All anomaly scores:", scores)


Iteration 0 Objective J = 88.5466934624949
Iteration 1 Objective J = 88.54669346249395
Converged at iteration 1
Sample 0 anomaly score: 1.2263245635028446
Sample 1 anomaly score: 1.0273585779436536
Sample 2 anomaly score: 1.7334552325115056
Sample 3 anomaly score: 1.4533322416212353
Sample 4 anomaly score: 1.588197085357677
Sample 5 anomaly score: 5.582402484840623
Sample 6 anomaly score: 5.728138638741637
Sample 7 anomaly score: 5.362384845175358
Sample 8 anomaly score: 5.178467246536957
Sample 9 anomaly score: 2.4350265342973003
Sample 10 anomaly score: 3.655393402246609
Sample 11 anomaly score: 7.188430032920555
Sample 12 anomaly score: 5.42610385715881
Sample 13 anomaly score: 5.590523894126414
Sample 14 anomaly score: 4.162184815272492
Sample 15 anomaly score: 10.417938798972472
Sample 16 anomaly score: 11.367496336104425
Sample 17 anomaly score: 7.786537599481928
Sample 18 anomaly score: 7.043511422763445
Sample 19 anomaly score: 5.6799229461940115
Sample 20 anomaly score: 5.8690